In [46]:
import os
import pandas as pd
from datetime import timedelta

data = pd.read_csv("../../data/supermarket_sales - Sheet1.csv")
print(data)

data = data[["Branch", "Product line", "Date", "Quantity"]]
data["Date"] = pd.to_datetime(data["Date"])
print(data)

min_date = data["Date"].min()
max_date = data["Date"].max()
delta = max_date - min_date

for i in range(delta.days + 1):
    day = min_date + timedelta(days=i)
    for branch in pd.unique(data["Branch"]):
        for product_line in pd.unique(data["Product line"]):
            data.loc[len(data)] = [branch, product_line, day, 0]
data = data.groupby(["Branch", "Product line", "Date"])["Quantity"].sum().reset_index()
print(data)
data.to_pickle("model_data.pkl")

      Invoice ID Branch       City Customer type  Gender  \
0    750-67-8428      A     Yangon        Member  Female   
1    226-31-3081      C  Naypyitaw        Normal  Female   
2    631-41-3108      A     Yangon        Normal    Male   
3    123-19-1176      A     Yangon        Member    Male   
4    373-73-7910      A     Yangon        Normal    Male   
..           ...    ...        ...           ...     ...   
995  233-67-5758      C  Naypyitaw        Normal    Male   
996  303-96-2227      B   Mandalay        Normal  Female   
997  727-02-1313      A     Yangon        Member    Male   
998  347-56-2442      A     Yangon        Normal    Male   
999  849-09-3807      A     Yangon        Member  Female   

               Product line  Unit price  Quantity   Tax 5%      Total  \
0         Health and beauty       74.69         7  26.1415   548.9715   
1    Electronic accessories       15.28         5   3.8200    80.2200   
2        Home and lifestyle       46.33         7  16.2155  

In [35]:
import os
import pandas as pd

data = pd.read_pickle("./model_data.pkl")
print(data)

train_data = pd.DataFrame(columns=["Branch", "Product line", "Month"] + [f"{i}" for i in range(14)] + ["Class"])
branches = pd.unique(data["Branch"])
product_lines = pd.unique(data["Product line"])

for branch in branches:
    for product_line in product_lines:
        cur_frame = data[(data["Branch"] == branch) & (data["Product line"] == product_line)]
        values = data["Quantity"]
        cur_slide = values.iloc[:14].to_list()
        for i in range(14, values.shape[0]):
            label = 1
            
            if cur_slide[-1] - values[i] >= cur_slide[-1] * 0.33:
                label = 0
            elif cur_slide[-1] - values[i] <= cur_slide[-1] * -0.33:
                label = 2
            train_data.loc[len(train_data.index)] = [branch, product_line, data.iloc[i]["Date"].month] + cur_slide + [label]
            
            cur_slide.pop(0)
            cur_slide.append(values[i])
            
print(train_data)
train_data.to_pickle("transformed_model_data.pkl")

     Branch            Product line       Date  Quantity
0         A  Electronic accessories 2019-01-01        10
1         A  Electronic accessories 2019-01-02         0
2         A  Electronic accessories 2019-01-03         0
3         A  Electronic accessories 2019-01-04         0
4         A  Electronic accessories 2019-01-05         7
...     ...                     ...        ...       ...
1597      C       Sports and travel 2019-03-26         0
1598      C       Sports and travel 2019-03-27         0
1599      C       Sports and travel 2019-03-28         0
1600      C       Sports and travel 2019-03-29         0
1601      C       Sports and travel 2019-03-30         0

[1602 rows x 4 columns]
      Branch            Product line  Month   0  1  2  3  4  5  6   7   8   9  \
0          A  Electronic accessories      1  10  0  0  0  7  0  4   7   0   0   
1          A  Electronic accessories      1   0  0  0  7  0  4  7   0   0   0   
2          A  Electronic accessories      1   0 

In [33]:
import os
import pandas as pd

data = pd.read_pickle("./transformed_model_data.pkl")
#print(data)
train_data = data.sample(frac=0.8)
test_data = data.drop(train_data.index)

train_0 = train_data[train_data["Class"] == 0]
train_1 = train_data[train_data["Class"] == 1]
train_2 = train_data[train_data["Class"] == 2]
max_train = (train_0 if train_0.shape[0] >= train_1.shape[0] and train_0.shape[0] >= train_2.shape[0] else
             train_1 if train_1.shape[0] >= train_0.shape[0] and train_1.shape[0] >= train_2.shape[0] else
             train_2)

test_0 = test_data[test_data["Class"] == 0]
test_1 = test_data[test_data["Class"] == 1]
test_2 = test_data[test_data["Class"] == 2]
max_test = (test_0 if test_0.shape[0] >= test_1.shape[0] and test_0.shape[0] >= test_2.shape[0] else
             test_1 if test_1.shape[0] >= test_0.shape[0] and test_1.shape[0] >= test_2.shape[0] else
             test_2)

for data_item in [train_0, train_1, train_2]:
    cur_num = data_item.shape[0]
    while cur_num + data_item.shape[0] < max_train.shape[0]:
        train_data = pd.concat([train_data, data_item])
        cur_num += data_item.shape[0]

for data_item in [test_0, test_1, test_2]:
    cur_num = data_item.shape[0]
    while cur_num + data_item.shape[0] < max_test.shape[0]:
        test_data = pd.concat([test_data, data_item])
        cur_num += data_item.shape[0]
        
print((data["Class"] == 0).sum() / data.shape[0])
print((data["Class"] == 1).sum() / data.shape[0])
print((data["Class"] == 2).sum() / data.shape[0])
print()

print((train_data["Class"] == 0).sum() / train_data.shape[0])
print((train_data["Class"] == 1).sum() / train_data.shape[0])
print((train_data["Class"] == 2).sum() / train_data.shape[0])
print()

print((test_data["Class"] == 0).sum() / test_data.shape[0])
print((test_data["Class"] == 1).sum() / test_data.shape[0])
print((test_data["Class"] == 2).sum() / test_data.shape[0])
print()

print(pd.merge(train_data, test_data).shape[0])
print()

train_data.to_pickle("train_data.pkl")
test_data.to_pickle("train_data.pkl")

0.6032745591939547
0.0535264483627204
0.3431989924433249

0.3933137796508813
0.38371728792322807
0.22296893242589058

0.39146800501882056
0.38268506900878296
0.2258469259723965

0



In [ ]:
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tenforflow.keras import layers

train_data = pd.read_pickle("./train_data.pkl")
test_data = pd.read_pickle("./test_data.pkl")

